# Modelo VGG16 Preentrenado para Clasificación de Imágenes

## Introducción
En esta lección, aprenderemos a usar el modelo VGG16 preentrenado para la tarea de clasificación de imágenes. VGG16 es una arquitectura profunda de red neuronal que demostró ser efectiva para reconocer imágenes.

## Objetivo
El objetivo de este cuaderno es construir un modelo de clasificación de imágenes que clasifique imágenes de gatos y perros utilizando transfer learning.

In [ ]:
# Importamos TensorFlow, la principal biblioteca para Deep Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import VGG16 # Para cargar el modelo VGG16
from tensorflow.keras.preprocessing import image # Para cargar imágenes
from tensorflow.keras.models import Model # Para construir nuestro nuevo modelo
from tensorflow.keras.layers import Dense, Flatten # Para añadir capas a nuestro modelo
import numpy as np # Para manipular arreglos de números
import matplotlib.pyplot as plt # Para mostrar imágenes

##Carga del Modelo VGG16


In [ ]:
# Cargamos el modelo VGG16 preentrenado en el conjunto de datos ImageNet.
# 'weights="imagenet"': Usa los pesos entrenados en ImageNet.
# 'include_top=False': Quitamos la capa final de clasificación para que podamos añadir la nuestra.
# 'input_shape=(224, 224, 3)': Le decimos al modelo el tamaño de las imágenes que va a esperar (altura, anchura, canales de color RGB).
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Opcional: Mostramos un resumen del modelo para ver sus capas (no es necesario entenderlo todo)
# base_model.summary()
print("Modelo VGG16 cargado exitosamente. Es la 'base de conocimiento' de nuestro experto.")

Modelo VGG16 cargado exitosamente. Es la 'base de conocimiento' de nuestro experto.


##Preparación del Modelo


In [ ]:
# Congelamos las capas del modelo base para que no se reentrenen.
# Así, solo ajustaremos las nuevas capas que añadiremos.
base_model.trainable = False
print("Capas del modelo base congeladas. ¡Listo para añadir nuestra parte específica!")

Capas del modelo base congeladas. ¡Listo para añadir nuestra parte específica!


In [ ]:
# Creamos una nueva capa que 'aplanará' la salida del modelo base.
# Esto es como tomar toda la información visual del experto y convertirla en una lista de números.
x = Flatten()(base_model.output)

# Añadimos una capa densa (o completamente conectada) con activación 'relu'.
# Esta capa aprenderá patrones complejos de la información aplanada.
x = Dense(256, activation='relu')(x)

# Añadimos la capa final de salida.
# Si tenemos 2 clases (ej. perro/gato), usaremos 'Dense(2)' y activación 'sigmoid' (para clasificación binaria).
# Si tuviéramos N clases (ej. perro, gato, pájaro, caballo), usaríamos 'Dense(N)' y activación 'softmax'.
output = Dense(2, activation='sigmoid')(x) # Asumiendo 2 clases: Perro y Gato

# Creamos el modelo final, combinando el modelo base y nuestras nuevas capas.
model = Model(inputs=base_model.input, outputs=output)

print("Nuevas capas añadidas. ¡Nuestro experto ya tiene una nueva 'cabeza' para nuestra tarea!")
# model.summary() # Puedes descomentar para ver el resumen del modelo completo

Nuevas capas añadidas. ¡Nuestro experto ya tiene una nueva 'cabeza' para nuestra tarea!


##Preparación del Dataset

In [ ]:
import os
# Crea las carpetas para el dataset
os.makedirs('data/train/cats', exist_ok=True)
os.makedirs('data/train/dogs', exist_ok=True)
print("Carpetas 'data/train/cats' y 'data/train/dogs' creadas.")

Carpetas 'data/train/cats' y 'data/train/dogs' creadas.


In [ ]:
# Definimos el tamaño de las imágenes que VGG16 espera
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32 # Número de imágenes que procesaremos a la vez (puedes probar con 4 o 8 si tu dataset es muy pequeño)

# Cargamos las imágenes desde nuestras carpetas.
# 'labels="inferred"' significa que Keras deduce las etiquetas de los nombres de las carpetas.
# 'image_size' redimensiona las imágenes.
# 'batch_size' para procesar en grupos.
train_ds = tf.keras.utils.image_dataset_from_directory(
    './data/train',
    labels='inferred',
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE
)

print(f"Dataset de entrenamiento cargado. Clases encontradas: {train_ds.class_names}")

# Opcional: Visualiza algunas imágenes del dataset para verificar
# plt.figure(figsize=(10, 10))
# for images, labels in train_ds.take(1):
#     for i in range(min(9, len(images))): # Muestra hasta 9 imágenes
#         ax = plt.subplot(3, 3, i + 1)
#         plt.imshow(images[i].numpy().astype("uint8"))
#         plt.title(train_ds.class_names[labels[i]])
#         plt.axis("off")
# plt.show()

 ## Compilación y Entrenamiento del Modelo

In [ ]:
# Compilamos el modelo.
# 'optimizer="adam"': El algoritmo que ajustará los pesos de las capas nuevas.
# 'loss="binary_crossentropy"': La función que mide qué tan bien lo está haciendo el modelo (para 2 clases).
# 'metrics=["accuracy"]': La métrica que queremos ver durante el entrenamiento (qué tan preciso es).
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

print("Modelo compilado y listo para entrenar.")

In [ ]:
# Entrenamos el modelo.
# 'train_ds': Nuestros datos de entrenamiento.
# 'epochs=5': El número de veces que el modelo verá todo el dataset.
# ¡Experimenten con el número de epochs!
history = model.fit(train_ds, epochs=5)

print("\nEntrenamiento finalizado.")

##Evaluación del Modelo

In [ ]:
# URL de una imagen de prueba (pueden cambiarla por una propia que suban a Colab)
# Ejemplo: una URL de una imagen de gato o perro que encuentren online
img_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/1/15/Cat_August_2010-4.jpg/1200px-Cat_August_2010-4.jpg" # URL de un gato
# img_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/a/a2/Canis_lupus_familiaris_%28dog%29.jpg/1200px-Canis_lupus_familiaris_%28dog%29.jpg" # URL de un perro

# Descargamos la imagen (si es una URL) o la cargamos localmente
try:
    img_path = tf.keras.utils.get_file('test_image.jpg', origin=img_url)
except Exception:
    print("No se pudo descargar la imagen, asegúrate que la URL es correcta.")
    # O si ya la subieron manualmente a Colab, usen su ruta:
    # img_path = '/content/tu_imagen_test.jpg' # Ajusta esta ruta

# Cargamos la imagen y la redimensionamos al tamaño esperado por el modelo
img = image.load_img(img_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) # Añadimos una dimensión extra para el batch
img_array /= 255.0 # Normalizamos los valores de píxeles (0-1)

# Realizamos la predicción
predictions = model.predict(img_array)

# Mostramos la imagen y la predicción
plt.imshow(img)
plt.axis('off')

class_names = train_ds.class_names # Las clases que encontró tu dataset (ej. ['cats', 'dogs'])

# Interpretamos la predicción
# Si tienes 2 clases y sigmoid, la salida es un valor entre 0 y 1.
# Si es >0.5 es la clase 1, si es <0.5 es la clase 0.
predicted_class_index = (predictions[0] > 0.5).astype(int)[0]
predicted_class_name = class_names[predicted_class_index]
confidence = predictions[0][0] if predicted_class_index == 1 else (1 - predictions[0][0])

plt.title(f"Predicción: {predicted_class_name} (Confianza: {confidence:.2f})")
plt.show()

print(f"Predicción numérica (es un valor entre 0 y 1 para la primera clase): {predictions[0]}")
print(f"Clases del dataset: {class_names}")